In [1]:
import nltk
import gensim
import rpy2

In [2]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
readRDS = robjects.r['readRDS']
df = readRDS('F:/IASD - PSL/PSL Week/NLP_lecture_PSLWeek-main/Data/UD.rds')
df = pandas2ri.rpy2py_dataframe(df)

In [ ]:
#df

In [71]:
df['sentence'][1]

"J'ai accouché en février 2019 et ma fille n'est pas encore inscrite à la CAF."

In [4]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
# Tokenization of each document
tokenized_doc = []
for d in df['sentence'].values:
    tokenized_doc.append(word_tokenize(d.lower()))
#tokenized_doc

In [ ]:
tokenized_doc[3]

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') + stopwords.words('french')
#stop_words=[word.decode('utf-8') for word in (stopwords.words('french'))]
#stop_words

In [17]:
#filtered_sentence=[]
filtered_tokens=[]
for doc in tokenized_doc:
    filtered_sentence=[w for w in doc if not (w in stop_words or w==".")]
    filtered_tokens.append(filtered_sentence)

In [69]:
prompt=filtered_tokens[1]
prompt

["j'ai",
 'accouché',
 'février',
 '2019',
 'fille',
 "n'est",
 'encore',
 'inscrite',
 'caf']

In [ ]:
# Convert tokenized document into gensim formated tagged data
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(filtered_tokens)]
tagged_data[5]

In [ ]:
model = Doc2Vec(vector_size=100, dbow_words= 1, dm=0, window=5, seed=1337, min_count=5, workers=4,alpha=0.025, min_alpha=0.025)
model.build_vocab(tagged_data)
for epoch in range(10):
    print("epoch "+str(epoch))
    model.train(tagged_data,total_examples=model.corpus_count, epochs=1)
    model.save('F:\IASD - PSL\PSL Week\Hackathon\doc2vec.model')
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay

In [5]:
doc2vec_model = Doc2Vec.load('F:\IASD - PSL\PSL Week\Hackathon\doc2vec.model')

In [6]:
life_situations=['mariage','pacs','concubinage','marier','pacser','épouser','civil','famille','divorce','séparation','civil','naissance','enfant','emploi','travail','salarié','embaucher']

In [68]:
sit={}
for i in range(len(life_situations)):
    try:
        sim=doc2vec_model.wv.similarity(prompt,life_situations[i])
        #print(life_situations[i])
        #print(max(sim))
        x=max(sim)
        if x>=0.04:
            sit[life_situations[i]]=max(sim)
    except:
        pass
print(sit)

{'mariage': 0.14669679, 'pacs': 0.09909782, 'concubinage': 0.13631359, 'marier': 0.12836395, 'pacser': 0.08915599, 'civil': 0.09938142, 'famille': 0.09757103, 'divorce': 0.14079091, 'séparation': 0.08748729, 'naissance': 0.17686586, 'enfant': 0.12258239, 'emploi': 0.085410446, 'travail': 0.09822284, 'salarié': 0.11018398, 'embaucher': 0.1091334}


In [106]:
def predict_situation(prompt):
    tokenized_doc.append(word_tokenize(d.lower()))
    filtered_sentence=[w for w in doc if not (w in stop_words or w==".")]
    sit={}
    for i in range(len(life_situations)):
        try:
            sim=doc2vec_model.wv.similarity(filtered_sentence,life_situations[i])
        #print(life_situations[i])
        #print(max(sim))
            x=max(sim)
            if x>=0.4:
                sit[life_situations[i]]=max(sim)
            else:
                pass
        except:
            pass
    if sit=={}:
        print('prompt not in life situations')
    else:
        maximum = max(sit, key=sit.get)
        print(maximum, round(sit[maximum],2))
    

In [107]:
prompt="J'ai accouché en février 2019 et ma fille n'est pas encore inscrite à la CAF"


In [108]:
predict_situation(prompt)

prompt not in life situations
